<a href="https://colab.research.google.com/github/kesavsivakumar/data_engineer_assessment_first_round_2024/blob/main/DATA_ENGINEER_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###setting up spark in colab environment
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ["HADOOP_HOME"] = os.environ["SPARK_HOME"]

In [3]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [4]:
#!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=4be98bb0bc4c66ff3da76c3ba70397b74c5ed717ab67f0f91825ab7d33a3c265
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [5]:
import findspark
findspark.init()

In [6]:
findspark.find()

'/content/spark-3.0.0-bin-hadoop3.2'

In [8]:
from pyspark.sql import SparkSession
##creating the spark session
spark = SparkSession.builder.master("local").appName("DE_assignment").config('spark.ui.port', '4050').getOrCreate()

In [12]:
##read json file to rdd
df_l1 = spark.read.option("multiline","true").json("/content/user_click_data.json.json")

In [13]:
df_l1.show()

+--------------+------------+--------------+-------+------+---------------+-------------------+--------------------+-------+
|       browser|        city|click_event_id|country|device|     ip_address|          timestamp|                 url|user_id|
+--------------+------------+--------------+-------+------+---------------+-------------------+--------------------+-------+
|Firefox Mobile|      Jaipur|           843|  India|Mobile| 240.246.150.88|2023-05-20 08:45:39|www.globalmart.co...| 525707|
|Firefox Mobile|      Jaipur|           844|  India|Mobile| 240.246.150.88|2023-05-20 08:49:34|www.globalmart.co...| 525707|
|        Safari|Philadelphia|           905|    USA|Mobile| 46.147.149.245|2023-05-20 08:51:44|www.globalmart.co...| 525679|
|        Safari|Philadelphia|           906|    USA|Mobile| 46.147.149.245|2023-05-20 08:56:27|www.globalmart.co...| 525679|
|        Safari|    Zaragoza|             2|  Spain|Mobile|  59.243.217.35|2023-05-20 08:33:31|www.globalmart.co...| 525616|


In [14]:
df_l1.count()

1004

In [15]:
df_l1.dtypes

[('browser', 'string'),
 ('city', 'string'),
 ('click_event_id', 'bigint'),
 ('country', 'string'),
 ('device', 'string'),
 ('ip_address', 'string'),
 ('timestamp', 'string'),
 ('url', 'string'),
 ('user_id', 'string')]

2.	The final output DataFrame should contain the following:


*   The average time (in minutes) users from each country spent on each web page (URL) for each date.
*   The total number of unique users from each country who visited each web page (URL) for each date.

*   The total number of clicks each web page (URL) garnered from each country for each date.





In [16]:
from pyspark.sql.functions import *

In [18]:
df_l1.createOrReplaceTempView("df")

In [59]:
qry2 ="Select country ,url,event_date , count() over(partition by url,country,event_date) as click_count ,avg(minutes_spent) over (partition by  country , url,event_date) as average_minutes_spent from (select distinct user_id,url,country ,cast(timestamp as DATE) as event_date, (unix_timestamp(max(timestamp) over(partition by user_id,URL,city,country,ip_address,device,browser))  - unix_timestamp( min(timestamp) over(partition by user_id,URL,city,country,ip_address,device,browser) ))/60  as minutes_spent from df)x"
qry_str_1 ='select distinct user_id,country,url,CAST(timestamp as date) as event_date ,(unix_timestamp(max(timestamp) over(partition by user_id))       - unix_timestamp( min(timestamp) over(partition by user_id) ))/60  as average_minutes_spent,count( user_id) over (partition by country,url,event_Date) as unique_user_count  from df '
fin1 = spark.sql(qry2)

In [60]:
fin1.show()

+-------+--------------------+----------+-----------+---------------------+
|country|                 url|event_date|click_count|average_minutes_spent|
+-------+--------------------+----------+-----------+---------------------+
|  India|www.globalmart.co...|2023-05-20|          0|    2.904166666666667|
|  India|www.globalmart.co...|2023-05-20|          0|    2.904166666666667|
|  India|www.globalmart.co...|2023-05-20|          0|    2.904166666666667|
|  India|www.globalmart.co...|2023-05-20|          0|    2.904166666666667|
|    USA|www.globalmart.co...|2023-05-20|          0|                  0.0|
|    USA|www.globalmart.co...|2023-05-20|          0|                  0.0|
|    USA|www.globalmart.co...|2023-05-20|          0|                  0.0|
|    USA|www.globalmart.co...|2023-05-20|          0|                  0.0|
|  Italy|www.globalmart.co...|2023-05-20|          0|                  0.0|
|  Italy|www.globalmart.co...|2023-05-20|          0|                  0.0|
|  Italy|www

In [44]:
fin1.count()


950

In [79]:
qry2=     """ select distinct country,
  URL,event_date,avg(minutes_spent) over (partition by url,country) as average_minutes_spent  from
  ( select distinct user_id,country,url,cast(timestamp as DATE) as event_date,
 (unix_timestamp(max(timestamp) over(partition by user_id,URL,country,cast(timestamp as DATE)))
 - unix_timestamp( min(timestamp) over(partition by user_id,URL,country,cast(timestamp as DATE)) ))/60  as minutes_spent from df) x """

In [80]:
fin2= spark.sql(qry2)

In [81]:
fin2.show()

+-------+--------------------+----------+---------------------+
|country|                 URL|event_date|average_minutes_spent|
+-------+--------------------+----------+---------------------+
|    USA|www.globalmart.co...|2023-05-20|                  0.0|
|    USA|www.globalmart.co...|2023-05-20|   2.6166666666666667|
|  India|www.globalmart.co...|2023-05-20|                  0.0|
|  Spain|www.globalmart.co...|2023-05-20|                  0.0|
|  Italy|www.globalmart.co...|2023-05-20|                  0.0|
|  Spain|www.globalmart.co...|2023-05-20|                  0.0|
|  India|www.globalmart.co...|2023-05-20|                  0.0|
|  Italy|www.globalmart.co...|2023-05-20|                  0.0|
|  India|www.globalmart.co...|2023-05-20|                  0.0|
|    USA|www.globalmart.co...|2023-05-20|                  0.0|
|England|www.globalmart.co...|2023-05-20|                  0.0|
|  Spain|www.globalmart.co...|2023-05-20|                  0.0|
|  Spain|www.globalmart.co...|2023-05-20

In [82]:
fin2.count()

241

In [104]:
qry3=     """ select distinct country,
  URL,event_date,count(user_id) over (partition by url,country,event_date) as unique_user_count  from
  ( select distinct user_id,country,url,cast(timestamp as DATE) as event_date from df) x """

In [105]:
fin3 = spark.sql(qry3)

In [106]:
fin3.show()

+-------+--------------------+----------+-----------------+
|country|                 URL|event_date|unique_user_count|
+-------+--------------------+----------+-----------------+
|  India|www.globalmart.co...|2023-05-20|                4|
|    USA|www.globalmart.co...|2023-05-20|                4|
|  Italy|www.globalmart.co...|2023-05-20|                3|
|  India|www.globalmart.co...|2023-05-20|                1|
|  Spain|www.globalmart.co...|2023-05-20|                3|
|  Spain|www.globalmart.co...|2023-05-20|                2|
|    USA|www.globalmart.co...|2023-05-20|                4|
|  India|www.globalmart.co...|2023-05-20|                9|
|England|www.globalmart.co...|2023-05-20|                2|
|England|www.globalmart.co...|2023-05-20|                4|
|  India|www.globalmart.co...|2023-05-20|                4|
|  India|www.globalmart.co...|2023-05-20|                5|
|    USA|www.globalmart.co...|2023-05-20|                1|
|  India|www.globalmart.co...|2023-05-20

In [107]:
fin3.count()

241

In [108]:
qry4=     """ select distinct  country,
  URL,event_date,count(user_id) over (partition by url,country,event_date) as click_count  from
  ( select  user_id,country,url,cast(timestamp as DATE) as event_date from df) x """

In [109]:
fin4 = spark.sql(qry4)


In [110]:
fin4.show()

+-------+--------------------+----------+-----------+
|country|                 URL|event_date|click_count|
+-------+--------------------+----------+-----------+
|  India|www.globalmart.co...|2023-05-20|          6|
|    USA|www.globalmart.co...|2023-05-20|          4|
|  Italy|www.globalmart.co...|2023-05-20|          3|
|  India|www.globalmart.co...|2023-05-20|          1|
|  Spain|www.globalmart.co...|2023-05-20|          3|
|  Spain|www.globalmart.co...|2023-05-20|          2|
|    USA|www.globalmart.co...|2023-05-20|          4|
|  India|www.globalmart.co...|2023-05-20|         11|
|England|www.globalmart.co...|2023-05-20|          2|
|England|www.globalmart.co...|2023-05-20|          4|
|  India|www.globalmart.co...|2023-05-20|          4|
|  India|www.globalmart.co...|2023-05-20|          5|
|    USA|www.globalmart.co...|2023-05-20|          1|
|  India|www.globalmart.co...|2023-05-20|          1|
|    USA|www.globalmart.co...|2023-05-20|          1|
|England|www.globalmart.co..

In [111]:
fin4.count()

241

In [134]:
fin_df = fin2.join(fin3,  (fin2.country == fin3.country) & (fin2.URL == fin3.URL) & (fin2.event_date == fin3.event_date),
               "inner").select (fin2.country,fin2.URL,fin2.event_date,fin2.average_minutes_spent,fin3.unique_user_count)

In [135]:
fin_df.show()

+-------+--------------------+----------+---------------------+-----------------+
|country|                 URL|event_date|average_minutes_spent|unique_user_count|
+-------+--------------------+----------+---------------------+-----------------+
|  India|www.globalmart.co...|2023-05-20|    6.816666666666666|                4|
|    USA|www.globalmart.co...|2023-05-20|                  0.0|                4|
|  Italy|www.globalmart.co...|2023-05-20|                  0.0|                3|
|  India|www.globalmart.co...|2023-05-20|                  0.0|                1|
|  Spain|www.globalmart.co...|2023-05-20|                  0.0|                3|
|  Spain|www.globalmart.co...|2023-05-20|                  0.0|                2|
|    USA|www.globalmart.co...|2023-05-20|                  0.0|                4|
|  India|www.globalmart.co...|2023-05-20|   3.5500000000000003|                9|
|England|www.globalmart.co...|2023-05-20|                  0.0|                2|
|England|www.glo

In [136]:
fin_df.count()

241

In [138]:
fin_df = fin_df.join(fin4,  (fin_df.country == fin4.country) & (fin_df.URL == fin4.URL) & (fin_df.event_date == fin4.event_date),
               "inner").select (fin_df.country,fin_df.URL,fin_df.event_date,fin_df.average_minutes_spent,fin_df.unique_user_count,fin4.click_count)

In [140]:
fin_df.show()

+-------+--------------------+----------+---------------------+-----------------+-----------+
|country|                 URL|event_date|average_minutes_spent|unique_user_count|click_count|
+-------+--------------------+----------+---------------------+-----------------+-----------+
|  India|www.globalmart.co...|2023-05-20|    6.816666666666666|                4|          6|
|    USA|www.globalmart.co...|2023-05-20|                  0.0|                4|          4|
|  Italy|www.globalmart.co...|2023-05-20|                  0.0|                3|          3|
|  India|www.globalmart.co...|2023-05-20|                  0.0|                1|          1|
|  Spain|www.globalmart.co...|2023-05-20|                  0.0|                3|          3|
|  Spain|www.globalmart.co...|2023-05-20|                  0.0|                2|          2|
|    USA|www.globalmart.co...|2023-05-20|                  0.0|                4|          4|
|  India|www.globalmart.co...|2023-05-20|   3.55000000000000

In [141]:
fin_df.count()

241

In [143]:
x = fin_df.toPandas()

In [144]:
x.to_csv('sample.csv')